## Part of Speech Tagging and Hidden Markov Models

### POS Tagging

Part of Speech (POS) Tag describes semantic function of a word in a sentence. Noun, adjective, verb etc. are all POS tags in English. Their annotation (or tagging) can be used for named entity recognition, coreference resolution, and attributing word probabilities in speech recognition. Below is an example of POS tagging.

<img src="images/pos.PNG" style="zoom: 70%;" />


### Markov Chains

Markov chains are probabilistic models that encodes transition probabilities between states. They are visually represented as a graph, where each node corresponds to a state and each arc denotes a transition that is annotated with a probability. Below is a toy markov chain that encodes verbs are more likely to be followed by a noun, rather than another verb. 

<img src="images/markov.png" style="zoom: 30%" />

We can represent a sentence as a sequence of POS tags and use a markov chain to predict the POS tag of words. Assume that we have a markov chain as follows:

<img src="images/markov_pos_tags.png" style="zoom: 40%" />

We can read this chain as with 0.2 probability, a noun is followed by a verb and with a 0.6 probability a noun is followed by a POS tag that is neither verb nor noun. So, we can decide the POS tag of the following verb with a stochastic (i.e. random) assignment. We also add a $\pi$ state to Markov Chain to obtain a probability estimation for the first word, since it does not have any preceeding tag.

**Remark:** Here, we inherently assume that POS tag of a word depends only on the POS tag of the immediately preceeding word. In other words, when the POS tag of the previous word is known, the rest of the preceeding wors is irrelevant for probabilty assignment. This is called **Markovian property**.


We can represent this chain with a matrix $A \in \mathbb{R}^{N+1\times N}$, where $N$ is the number of states. This is called the *transition matrix*. In matrix $A$, a cell $A_{ij}$ denotes the transition probabilty from state $i$ to state $j$. Therefore, row sums of $A$ is equal to $1$.

<img src="images/transition_table.png" style="zoom: 40%" />

### Hidden Markov Model

Hidden Markov Model (HMM) is a type of Markov Chain that assumes states are not observable but their outputs are. For POS tagging, it corresponds that we do not observe POS tags (hidden) but we observe their outputs, the words. This is a realistic approach since computers are not aware of the POS tags of words, but they can observe the words in the sentence. We can convert the previous chain to below HMM by adding observable states (words; drawn in rectangle) to the hidden ones (POS tags; drawn in dashed circle).

<img src="./images/hmm.png" style="zoom: 40%"/>

In an HMM, besides the transition probabilities, each state is associated with emission probabilities that denote the probability of each output at a state. The emission probabilities can again be stored in a matrix $B \in \mathbb{R}^{N\times|V|}$, where $|V|$ is the size of the vocabulary. The rows of $B$ also sums up to $1$

<img src="./images/emission_matrix.png" style="zoom: 40%" />

*Note: Last row does not sum up to $1$; which is an error.*

**Training** an HMM model means learning $A$ and $B$ matrices, which is quite straight-forward. Given a corpus with words and their POS tags, we count how many times a POS tag is followed by others to learn transition matrix, $A$. Similarly, we learn $B$ by counting how many times a word is tagged with each tag. We use smoothing for better generalization and add $\epsilon$ to each count and then normalize the counts to probabilties, by dividing each element with its row sum. 

**Prediction** using an HMM is more complicated. We use Viterbi algorithm, that computes the most probable sequence of hidden states, given the observations. For POS tagging, it computes the most likely sequence of POS tags of a sentence. 

**Viterbi algorithm** conssits of 3 steps which are *initialization*, *forward pass*, and *backward pass*. As the result, it outputs the probability of each tag for each word and the most likely sequence.

Viterbi uses two auxillary matrices $C, D \in \mathbb{R}^{N\times K}$, where $K$ is the number of words in the sentence. $C$ hold the optimal POS tag probabilities at each step whereas $D$ stores the indices of visited vertices. These matrices are initialized at the first step and populated column by column during forward pass. $C$ is populated according to following formula:

$$
c_{i, j}=\max _{k} c_{k, j-1} * a_{k, i} * b_{i, \text {cindex}\left(w_{j}\right)}
$$

We can break this formula into three parts. $ b_{i, \text{cindex}}(w_{j}) $ is the probability of tag $i$ emitting the word $k$, and $a_{k, i}$ is the transition probability from tag $k$ to tag $i$. Last, $c_{k, j-1}$ is the probability of the preceeding path. We compute this equation for each $k$ (tag/state) and find the maximum probability and store it into $C_{i,j}$. We also store $k$ to $D_{i,j}$ to find the most probable path during backward pass.

**Remark:** $c_{i,j}$ is the probability of the most probable path that tags word $j$ with tag $i$

During backward pass, we first find row index $i_1$ of the most probable path by finding max over the last column of $C$. We then go to cell $D_{i_1,k}$ and find the index $i_2$ of the state that led to here. We visit cell $D_{i_2, k-1}$ and iterate this way until we arrive in the first column. Below is an example backtracking on $D$ assuming that $i_1 = 1$. 

<img src="./images/backward_pass.png" style="zoom: 40%" />

With this path, we tag the word sequence as: $\pi \rightarrow t_2 \rightarrow t_3 \rightarrow t_1 \rightarrow t_3 \rightarrow t_1$, where $\pi$ is the artificial initial state